## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

2025-03-20 04:12:39.745944: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-20 04:12:40.954452: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
# class myModel:
#     def __init__(self):
#         ####################
#         '''声明模型对应的参数'''
#         self.W1 = tf.Variable(tf.random.normal([784, 128], mean=0.0, stddev=0.1))  # 第一层权重
#         self.b1 = tf.Variable(tf.zeros([128]))  # 第一层偏置
#         self.W2 = tf.Variable(tf.random.normal([128, 10], mean=0.0, stddev=0.1))  # 第二层权重
#         self.b2 = tf.Variable(tf.zeros([10]))  # 第二层偏置                           
#         ####################
#     def __call__(self, x):
#         ####################
#         '''实现模型函数体，返回未归一化的logits'''
#         x = tf.reshape(x, [-1, 784])          # 展平输入为 [batch_size, 784]
#         h = tf.matmul(x, self.W1) + self.b1   # 全连接层1
#         h = tf.nn.relu(h)                     # ReLU激活
#         logits = tf.matmul(h, self.W2) + self.b2 # 全连接层2
#         ####################
#         return logits
class myModel:
    def __init__(self):
        self.W1 = tf.Variable(shape=[28*28, 100], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[28*28, 100],
                                                         minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[100], dtype=tf.float32, initial_value=tf.zeros(100))
        self.W2 = tf.Variable(shape=[100, 10], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[100, 10],
                                                         minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]
    def __call__(self, x):
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
    
model = myModel()

optimizer = optimizers.Adam()

2025-03-20 04:12:42.013711: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(1000):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    if (epoch%100==0):
        print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3881056 ; accuracy 0.056183334
epoch 100 : loss 1.9185321 ; accuracy 0.59225
epoch 200 : loss 1.5431627 ; accuracy 0.71861666
epoch 300 : loss 1.2540581 ; accuracy 0.7604
epoch 400 : loss 1.0516932 ; accuracy 0.78815
epoch 500 : loss 0.9119846 ; accuracy 0.8077667
epoch 600 : loss 0.8127755 ; accuracy 0.8222833
epoch 700 : loss 0.73958504 ; accuracy 0.8343
epoch 800 : loss 0.68359727 ; accuracy 0.8423167
epoch 900 : loss 0.6394232 ; accuracy 0.84895
test loss 0.5841911 ; accuracy 0.8609
